In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from ydata_profiling import ProfileReport

In [2]:
import re

# Define a function to check for special characters
def has_special_characters(record):
    # Regular expression to match special characters
    return bool(re.search(r'[^a-zA-Z0-9\s]', record))


In [ ]:
def check_possible_dummyable_variables(data, max_unique_values=10):
    """
    Check for possible dummyable variables in the DataFrame and return a DataFrame
    with column names, their number of unique values and ammount of NaN.

    Parameters:
    - data: pd.DataFrame
        The DataFrame to check.
    - max_unique_values: int
        The maximum number of unique values for a variable to be considered dummyable.

    Returns:
    - dummyable_vars_df: pd.DataFrame
        A DataFrame with two columns: 'Column' (column names), 'UniqueValues' (number of unique values) and 'TotalNan' (number of Nan).
    """
    dummyable_vars = []

    for col in data.columns:
        if ((data[col].dtype == 'object') | (data[col].dtype == 'category')) and len(data[col].unique()) <= max_unique_values:
            nanValues = data[col].isna().sum()
            dummyable_vars.append({'Column': col, 'UniqueValues': len(data[col].unique()), 'TotalNan':  str(nanValues)})

    # Convert the list of dictionaries to a DataFrame
    dummyable_vars_df = pd.DataFrame(dummyable_vars)

    return dummyable_vars_df


In [38]:
def check_high_frequency_categories(data, min_unique_values=10):
    """
    Check for variables that have more than min_unique_values categories in the DataFrame and return a DataFrame
    with column names and their number of unique values.

    Parameters:
    - data: pd.DataFrame
        The DataFrame to check.
    - max_unique_values: int
        The maximum number of unique values for a variable to be considered dummyable.

    Returns:
    - dummyable_vars_df: pd.DataFrame
        A DataFrame with two columns: 'Column' (column names), 'UniqueValues' (number of unique values) and 'TotalNan' (number of Nan).
    """
    dummyable_vars = []

    for col in data.columns:
        if ((data[col].dtype == 'object') | (data[col].dtype == 'category')) and len(data[col].unique()) >= min_unique_values:
            nanValues = data[col].isna().sum()
            dummyable_vars.append({'Column': col, 'UniqueValues': len(data[col].unique()), 'TotalNan':  str(nanValues)})

    # Convert the list of dictionaries to a DataFrame
    dummyable_vars_df = pd.DataFrame(dummyable_vars)

    return dummyable_vars_df


# Inspección de variables
Cargar los documentos y revisar variables categóricas y numericas

In [12]:
df_train = pd.read_csv('training_values.csv', delimiter=',')
df_label = pd.read_csv('training_labels.csv', delimiter=',')
df_test = pd.read_csv('Test.csv', delimiter=',')

print(df_train.shape)
print(df_label.shape)
print('------')
print(df_test.shape)

(59400, 40)
(59400, 2)
------
(14850, 40)


In [ ]:
# Todo:
# Combinar df_pump y df_test para aplicar las transformaciones en ambos dataframes - crear una columna para identificar los dataframes de origen
# Validar Id's
# Verificar tipologia de variables
# Valores fuera de rango
# Eliminar las variables que no aportan información
# Verificar si hay valores nulos

In [13]:
print(df_train.columns)
df_train['original_file'] = 'train'
df_test['original_file'] = 'test'

print(df_train.columns)

# Concatenate the train and test DataFrames
# to apply transformations on both dataframes
# and create a column to identify the source DataFrame

df_pump = pd.concat([df_train, df_test], axis=0)
df_pump.info()

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group'],
      dtype='object')
Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management',

In [14]:
print(df_pump['id'].duplicated().value_counts())
print(df_label['id'].duplicated().value_counts())

id
False    74250
Name: count, dtype: int64
id
False    59400
Name: count, dtype: int64


In [8]:
print(df_label['id'].isin(df_pump['id']).value_counts())

id
True    59400
Name: count, dtype: int64


In [15]:
var_categoricas = df_pump.select_dtypes(include=['object','category']).columns
var_num = df_pump.select_dtypes(exclude=['object','category']).columns

In [10]:
df_pump.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,original_file
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,train
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,train
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,train
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,train
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,train


In [ ]:
# Analizar las variables de forma individual desde el reporte
pr = ProfileReport(df=df_pump)
pr.to_file('reporte.html')

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# Tipología de variables

In [16]:
# Clasifico las variables por conceptos: tipo date, personas, zona, pozo y costos - de acuerdo a alguna caracteristicas común
# Para analizar en caso de que se necesite hacer inferencia
tipo_date = ['date_recorded']
tipo_zona = ["wpt_name", "basin", "subvillage", "region", "lga", "ward", "gps_height", "longitude", "latitude", "region_code", "district_code"]
tipo_costos = ['payment','payment_type']
tipo_personas = ["funder", "installer", "public_meeting", "recorded_by", "scheme_management", "scheme_name", "management", "management_group"]
tipo_pozo = ["permit", "extraction_type", "extraction_type_group", "extraction_type_class", "water_quality", "quality_group", "quantity", "quantity_group", "source", "source_type", "source_class", "waterpoint_type", "waterpoint_type_group", "amount_tsh", "population", "construction_year"]



In [8]:
print(var_categoricas)
print(var_num)

Index(['date_recorded', 'funder', 'installer', 'wpt_name', 'basin',
       'subvillage', 'region', 'lga', 'ward', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'extraction_type',
       'extraction_type_group', 'extraction_type_class', 'management',
       'management_group', 'payment', 'payment_type', 'water_quality',
       'quality_group', 'quantity', 'quantity_group', 'source', 'source_type',
       'source_class', 'waterpoint_type', 'waterpoint_type_group',
       'original_file'],
      dtype='object')
Index(['id', 'amount_tsh', 'gps_height', 'longitude', 'latitude',
       'num_private', 'region_code', 'district_code', 'population',
       'construction_year'],
      dtype='object')


In [17]:
ver_a = check_possible_dummyable_variables(df_pump, 15)
ver_a.sort_values(by='UniqueValues', ascending=False)

,Column,UniqueValues,TotalNan
5,extraction_type_group,13,0
3,scheme_management,12,4847
7,management,12,0
15,source,10,0
0,basin,9,0
11,water_quality,8,0
9,payment,7,0
18,waterpoint_type,7,0
16,source_type,7,0
10,payment_type,7,0


In [20]:
#Analizar permit, public_meeting, scheme_management
# df_pump.recorded_by.value_counts(dropna=False)
#Eliminar recorded_by

#df_pump.permit.value_counts(dropna=False)
# reemplazar nan for unkown

# df_pump.public_meeting.value_counts(dropna=False, normalize=True)
# reemplazar nan for unkown

#df_pump.scheme_management.value_counts(dropna=False, normalize=True)
# reemplazar nan for unkown


recorded_by
GeoData Consultants Ltd    74250
Name: count, dtype: int64

In [24]:
# Gestionando los NaN
df_pump['permit'] = df_pump['permit'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['public_meeting'] = df_pump['public_meeting'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['scheme_management'] = df_pump['scheme_management'].apply(lambda x: 'Unknown' if pd.isna(x) else x)

In [28]:

# df_pump.extraction_type_group.value_counts(dropna=False, normalize=True)
# df_pump.scheme_management.value_counts(dropna=False, normalize=True)
# df_pump.management.value_counts(dropna=False, normalize=True)
df_pump.source.value_counts(dropna=False, normalize=True)


source
spring                  0.285737
shallow well            0.284714
machine dbh             0.186155
river                   0.161131
rainwater harvesting    0.038559
hand dtw                0.014923
lake                    0.012795
dam                     0.011313
other                   0.003515
unknown                 0.001158
Name: proportion, dtype: float64

In [29]:
# Agrupacion de valores raros
# Horizontal list
extraction_type_group_rare = ['india mark ii', 'afridev', 'rope pump', 'other handpump', 'wind-powered', 'other motorpump', 'india mark iii', 'swn 80', 'mono', 'other']
df_pump['extraction_type_group'] = np.where(df_pump['extraction_type_group'].isin(extraction_type_group_rare), 'Other minor extraction type', df_pump['extraction_type_group'])

scheme_management_rare = ['Water Board', 'WUA', 'Water authority', 'Parastatal', 'Company', 'Private operator', 'Other', 'SWC', 'Trust']
df_pump['scheme_management'] = np.where(df_pump['scheme_management'].isin(scheme_management_rare), 'Other minor scheme', df_pump['scheme_management'])

management_rare = ['private operator', 'parastatal', 'water authority', 'other', 'company', 'unknown', 'other - school', 'trust']
df_pump['management'] = np.where(df_pump['management'].isin(management_rare), 'Other minor Operator', df_pump['management'])

source_rare = ['rainwater harvesting', 'hand dtw', 'lake', 'dam', 'other', 'unknown']
df_pump['source'] = np.where(df_pump['source'].isin(source_rare), 'Other minor source', df_pump['source'])


In [31]:
# Eliminar recorded_by - No aporta informacion 
df_pump.drop(columns='recorded_by', inplace=True)

In [32]:
ver_b = check_possible_dummyable_variables(df_pump, 15)
ver_b.sort_values(by='UniqueValues', ascending=False)

,Column,UniqueValues,TotalNan
0,basin,9,0
10,water_quality,8,0
5,extraction_type_class,7,0
8,payment,7,0
9,payment_type,7,0
17,waterpoint_type,7,0
15,source_type,7,0
11,quality_group,6,0
18,waterpoint_type_group,6,0
14,source,5,0


In [34]:
tipo_personas = ["funder", "installer", "public_meeting", "scheme_management", "scheme_name", "management", "management_group"]

In [37]:
df_pump.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,original_file
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,train
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,Other minor source,rainwater harvesting,surface,communal standpipe,communal standpipe,train
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,Other minor source,dam,surface,communal standpipe multiple,communal standpipe,train
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,train
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,Other minor source,rainwater harvesting,surface,communal standpipe,communal standpipe,train


In [ ]:
# Procesar categorías que tienen mas de 15 únicos valores

In [39]:
ver_c = check_high_frequency_categories(df_pump, 15)
ver_c.sort_values(by='UniqueValues', ascending=False)


,Column,UniqueValues,TotalNan
3,wpt_name,45684,2
4,subvillage,21426,470
8,scheme_name,2868,36052
2,installer,2411,4532
1,funder,2140,4507
7,ward,2098,0
0,date_recorded,369,0
6,lga,125,0
5,region,21,0
9,extraction_type,18,0


In [ ]:
# Gestionar los NaN de las variables categoricas que tienen alta frecuencia de categorias

# df_pump.wpt_name.value_counts(dropna=False, normalize=True)
# 2 faltantes, reemplazar por unknown

# df_pump.subvillage.value_counts(dropna=False, normalize=True)
# Menor al 1%, reemplazar por unknown

# df_pump.scheme_name.value_counts(dropna=False, normalize=True)
# scheme_name y scheme management tienen la misma descripcion, ambas columnas representan la misma informacion
# Se puede utilizar la informacion de scheme management para llenar lo faltante de scheme_name (no óptima) o eliminar la columna,
# porque ya tenemos al informacion mejor organizada en la otra columna y hay gran cantidad de faltantes ~50% faltantes
# Se elimna la columna


# df_pump.installer.value_counts(dropna=False, normalize=True)
# Reemplazar faltantes con Unknown ~6%

# df_pump.funder.value_counts(dropna=False, normalize=True)
# Reemplazar faltantes con Unknown ~6%

funder
Government Of Tanzania    0.152175
NaN                       0.060700
Danida                    0.052620
Hesawa                    0.037468
Rwssp                     0.022936
                            ...   
Sido                      0.000013
Mosqure                   0.000013
Kkkt Canal                0.000013
Rc Missionary             0.000013
Velem                     0.000013
Name: proportion, Length: 2140, dtype: float64

In [48]:
df_pump['wpt_name'] = df_pump['wpt_name'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['subvillage'] = df_pump['subvillage'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['installer'] = df_pump['installer'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['funder'] = df_pump['funder'].apply(lambda x: 'Unknown' if pd.isna(x) else x)

df_pump.drop(columns='scheme_name', inplace=True)

In [49]:
ver_c = check_high_frequency_categories(df_pump, 15)
ver_c.sort_values(by='UniqueValues', ascending=False)


,Column,UniqueValues,TotalNan
3,wpt_name,45683,0
4,subvillage,21426,0
2,installer,2410,0
1,funder,2139,0
7,ward,2098,0
0,date_recorded,369,0
6,lga,125,0
5,region,21,0
8,extraction_type,18,0


In [46]:
df_pump[tipo_personas]

,funder,installer,public_meeting,scheme_management,scheme_name,management,management_group
0,Roman,Roman,True,VWC,Roman,vwc,user-group
1,Grumeti,GRUMETI,Unknown,Other minor scheme,NaN,wug,user-group
2,Lottery Club,World vision,True,VWC,Nyumba ya mungu pipe scheme,vwc,user-group
3,Unicef,UNICEF,True,VWC,NaN,vwc,user-group
4,Action In A,Artisan,True,Unknown,NaN,Other minor Operator,other
...,...,...,...,...,...,...,...
14845,Danida,Da,True,VWC,Bagamoyo wate,vwc,user-group
14846,Hiap,HIAP,True,VWC,NaN,vwc,user-group
14847,NaN,NaN,True,VWC,NaN,vwc,user-group
14848,Germany,DWE,True,VWC,Mradi wa maji wa maposeni,vwc,user-group


In [ ]:
# Crear Dummy columns para las variables categoricas que tienen poca frecuencia de categorias
# Reemplazar por la frecuencia a las variables categoricas que tienen alta frecuencia de categorias

# Revisar valores mal codificados

In [9]:
df_pump['subvillage'].value_counts(dropna=False, normalize=False).where(lambda x: x > 100).dropna().sort_values(ascending=False)
# .where(lambda x: x > 500).dropna().sort_values(ascending=False)

subvillage
Madukani      508.0
Shuleni       506.0
Majengo       502.0
Kati          373.0
NaN           371.0
Mtakuja       262.0
Sokoni        232.0
M             187.0
Muungano      172.0
Mbuyuni       164.0
Mlimani       152.0
Songambele    147.0
Msikitini     134.0
Miembeni      134.0
1             132.0
Kibaoni       114.0
Kanisani      111.0
I             109.0
Mapinduzi     109.0
Mjimwema      108.0
Mjini         108.0
Mkwajuni      104.0
Mwenge        102.0
Name: count, dtype: float64

In [16]:
var_zona = ['wpt_name',
'basin',
'subvillage',
'region',
'lga',
'ward',
'gps_height',
'longitude',
'latitude',
'region_code',
'district_code']
df_pump[df_pump['longitude'].isna() & df_pump['latitude'].isna()][var_zona].head(10)


,wpt_name,basin,subvillage,region,lga,ward,gps_height,longitude,latitude,region_code,district_code


In [24]:
df_pump[df_pump['longitude'] <= 0 ][var_zona]

,wpt_name,basin,subvillage,region,lga,ward,gps_height,longitude,latitude,region_code,district_code
21,Muungano,Lake Victoria,Ibabachegu,Shinyanga,Bariadi,Ikungulyabashashi,0,0.0,-2.000000e-08,17,1
53,Polisi,Lake Victoria,Center,Mwanza,Geita,Nyang'hwale,0,0.0,-2.000000e-08,19,6
168,Wvt Tanzania,Lake Victoria,Ilula,Shinyanga,Bariadi,Chinamili,0,0.0,-2.000000e-08,17,1
177,Kikundi Cha Wakina Mama,Lake Victoria,Mahaha,Shinyanga,Bariadi,Bunamhala,0,0.0,-2.000000e-08,17,1
253,Kwakisusi,Lake Victoria,Nyamatala,Mwanza,Magu,Malili,0,0.0,-2.000000e-08,19,2
...,...,...,...,...,...,...,...,...,...,...,...
59189,Wazazo,Lake Victoria,Mwamabuli,Shinyanga,Bariadi,Mhunze,0,0.0,-2.000000e-08,17,1
59208,Mtakuja,Lake Victoria,Mbiti,Shinyanga,Bariadi,Kinang'weli,0,0.0,-2.000000e-08,17,1
59295,Maendeleo,Lake Victoria,Mwamalizi,Shinyanga,Bariadi,Chinamili,0,0.0,-2.000000e-08,17,1
59324,Mwazwilo,Lake Victoria,Mbita,Shinyanga,Bariadi,Mbita,0,0.0,-2.000000e-08,17,1


In [50]:

df_pump['subvillage'] = df_pump['subvillage'].apply(lambda x: 'Unknown' if pd.isna(x) else x)
df_pump['subvillage'].value_counts(dropna=False, normalize=False)

subvillage
Madukani        508
Shuleni         506
Majengo         502
Kati            373
Unknown         371
               ... 
Kipompo           1
Chanyamilima      1
Ikalime           1
Kemagaka          1
Kikatanyemba      1
Name: count, Length: 19288, dtype: int64

In [65]:
df_pump[df_pump['subvillage'].apply(has_special_characters)]['subvillage']

5         Moa/Mwereme
59        Izimbya 'A'
114        Chang'Ombe
169            Ng'Uni
252      Maring'A Juu
             ...     
59074        Maring'A
59118         Meng'We
59244        Lung'Uda
59247    Wanging'Ombe
59321       King`Ombe
Name: subvillage, Length: 1097, dtype: object

In [37]:
df_subvillage_nonzero = df_pump[((df_pump['longitude'] != 0) & (df_pump['latitude'] != 0))][var_zona]

df_subvillage_nonzero.groupby(['subvillage']).agg(
    {
        'longitude': 'max',
        'latitude': 'min'
    }
)

,longitude,latitude
subvillage,,
##,31.460087,-1.553813
'A' Kati,33.730739,-1.903902
1,33.078106,-2.533517
18,32.134800,-4.582413
19,32.135162,-4.595656
...,...,...
Zumbawanu Shuleni,36.510212,-6.953036
Zunga,38.366623,-6.346883
Zunguni,38.837185,-7.144544


In [33]:
df_pump[(df_pump['subvillage'] == 'Mahaha') & ((df_pump['longitude'] != 0) | (df_pump['latitude'] != 0))][var_zona]

,wpt_name,basin,subvillage,region,lga,ward,gps_height,longitude,latitude,region_code,district_code
177,Kikundi Cha Wakina Mama,Lake Victoria,Mahaha,Shinyanga,Bariadi,Bunamhala,0,0.000000,-2.000000e-08,17,1
14010,Kwa Mzee Lupindo,Lake Nyasa,Mahaha,Ruvuma,Mbinga,Liuli,459,34.662079,-1.112878e+01,10,3
25817,Muungano,Lake Victoria,Mahaha,Shinyanga,Bariadi,Bumera,0,0.000000,-2.000000e-08,17,1
27383,Muungano,Lake Victoria,Mahaha,Shinyanga,Bariadi,Bunamhala,0,0.000000,-2.000000e-08,17,1
34787,Bupandamawe,Lake Victoria,Mahaha,Shinyanga,Bariadi,Bunamhala,0,0.000000,-2.000000e-08,17,1


In [83]:
field_to_check='installer'
df_pump[df_pump[field_to_check].str.len() <= 2][field_to_check].value_counts(dropna=False, normalize=False)

installer
0     777
WU    301
DW    246
Da    224
DH    202
     ... 
MW      1
R       1
Nu      1
WA      1
M       1
Name: count, Length: 109, dtype: int64

In [81]:
df_pump[df_pump['installer'] == 'RWE'][['funder', 'installer', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name','management',
       'management_group']]

,funder,installer,public_meeting,recorded_by,scheme_management,scheme_name,management,management_group
62,Norad,RWE,True,GeoData Consultants Ltd,VWC,Mkongoro One,vwc,user-group
70,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,NaN,NaN,vwc,user-group
77,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Tengeru gravity water supply,vwc,user-group
143,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Vugiro,vwc,user-group
173,Dhv,RWE,True,GeoData Consultants Ltd,NaN,NaN,vwc,user-group
...,...,...,...,...,...,...,...,...
59088,World Bank,RWE,True,GeoData Consultants Ltd,VWC,Mnyuzi water supply,vwc,user-group
59104,W.D & I.,RWE,True,GeoData Consultants Ltd,VWC,Mkata ri,vwc,user-group
59205,France,RWE,True,GeoData Consultants Ltd,VWC,Tangeni,vwc,user-group
59269,Government Of Tanzania,RWE,True,GeoData Consultants Ltd,VWC,Muriti Water Supply,vwc,user-group


Trabajar con la variable tipo Date - que se puede extraer rapidamente valores numericos

In [ ]:
df_pump['date_recorded'] = pd.to_datetime(df_pump['date_recorded'])
df_pump['year'] = df_pump['date_recorded'].dt.year
df_pump['month'] = df_pump['date_recorded'].dt.month
df_pump['day'] = df_pump['date_recorded'].dt.day
df_pump['day_of_week'] = df_pump['date_recorded'].dt.dayofweek
df_pump['day_of_week'].value_counts(normalize=True)


day_of_week
2    0.156330
4    0.148434
3    0.148401
1    0.143131
5    0.139646
0    0.139562
6    0.124495
Name: proportion, dtype: float64

In [26]:
df_pump['duracion']=(pd.to_datetime('2013-12-31') - df_pump['date_recorded']).dt.days

In [ ]:
#Separa actividades que son realizadas por personas o entidades
df_pump[['funder', 'installer', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name','management',
       'management_group']]

,funder,installer,wpt_name,public_meeting,recorded_by,scheme_management,scheme_name,management,management_group
0,Roman,Roman,none,True,GeoData Consultants Ltd,VWC,Roman,vwc,user-group
1,Grumeti,GRUMETI,Zahanati,NaN,GeoData Consultants Ltd,Other,NaN,wug,user-group
2,Lottery Club,World vision,Kwa Mahundi,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,vwc,user-group
3,Unicef,UNICEF,Zahanati Ya Nanyumbu,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group
4,Action In A,Artisan,Shuleni,True,GeoData Consultants Ltd,NaN,NaN,other,other
...,...,...,...,...,...,...,...,...,...
59395,Germany Republi,CES,Area Three Namba 27,True,GeoData Consultants Ltd,Water Board,Losaa Kia water supply,water board,user-group
59396,Cefa-njombe,Cefa,Kwa Yahona Kuvala,True,GeoData Consultants Ltd,VWC,Ikondo electrical water sch,vwc,user-group
59397,NaN,NaN,Mashine,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group
59398,Malec,Musa,Mshoro,True,GeoData Consultants Ltd,VWC,NaN,vwc,user-group


In [50]:
# Separado para analizar con variables que tienen que ver con zonas geográficas
df_pump[['wpt_name',
'basin',
'subvillage',
'region',
'lga',
'ward']]

,wpt_name,basin,subvillage,region,lga,ward
0,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi
1,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta
2,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika
3,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu
4,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi
...,...,...,...,...,...,...
59395,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi
59396,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo
59397,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala
59398,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu


date_recorded = fecha -> calcular dia, mes, año, duracion
funder - 1986 distintos
installer - 2145 d
wpt_name
basil
subvillage
region
lga
ward
public_meeting
recorded_by - unico valor - REMOVER
scheme_management
scheme_name
permit
extraction_type
extraction_type_group
extraction_type_class
management
management_group
payment
payment_type
water_quality
quality_group
quantity
quantity_group
source
source_type
source_class
waterpoint_type
waterpoint_type_group